## CS254 Final Project: May 1st
 gathering previous data preparation into singlke function
 trying to implement VGG as a feature extractor
 then use SVM as a classifier
 
 potentially need to reformat data?

In [1]:
    
    #initial imports
    import os
    import random
    import numpy as np
    import pandas as pd
    import json
    import keras
    from keras.models import Sequential
    from keras.utils import to_categorical
    from keras.preprocessing import image
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from tqdm import tqdm
    import keras
    from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
    from keras.layers.normalization import BatchNormalization
    import numpy as np
    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg16 import preprocess_input
    from keras.preprocessing.image import load_img
    from keras.preprocessing.image import img_to_array
    from keras.models import Model
    from pickle import dump
    import pickle
    %matplotlib inline

In [ ]:

######################################################################################################
#####################################################################################################
#getting data ready
#create a dataframe containing the names of all of the jpgs in the full dataset
fileList = []
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data'):
    if file.endswith(".jpg"):
        fileList.append(file)

df = pd.DataFrame(fileList)

#also creating an index to use later in extracting the img labels from the json files
index = df.index

#formatting the dataframe
df = df.rename(columns={0: "Id"}) 
df['labels'] = df.Id
df['labels'] = df['labels'].str.replace('.jpg','')
df['tower_wooden'] = ""
df['tower_lattice'] = ""
df['tower_tucohy'] = ""
index = df.index
 ######################################################################################################
#####################################################################################################
for file in os.listdir('C:/Users/miapi/Downloads/ttpla_dataset/sized_data'):
    if file.endswith(".json"):

        #load file as json object
        filePath = ("C:/Users/miapi/Downloads/ttpla_dataset/sized_data/"+file)
        f = open(filePath,)
        json_file = json.load(f)
        #get file namme without extension
        file2 = file.replace('.json','')

        #find designated jpeg (row in df)
        #rslt_df = df.loc[df['labels'] == file2]
        apples_indices = index[df['labels'] == file2]

        #parse json file to see if tower_wooden, tower_lattice, or tower_tucohy exists
        #add 0 or 1 to the corresponding dataframe row accordingly

        for element in json_file['shapes']:

            #tower wooden
            bool_wood = any(sd['label']=='tower_wooden' for sd in json_file['shapes'])  
            if bool_wood:
                df.at[apples_indices,'tower_wooden']= 1
            else:
                df.at[apples_indices,'tower_wooden']= 0 

            #tower lattice
            bool_lat = any(sd['label']=='tower_lattice' for sd in json_file['shapes']) 
            if bool_lat:
                df.at[apples_indices,'tower_lattice']= 1
            else:
                df.at[apples_indices,'tower_lattice']= 0

            #tower tucohy
            bool_tuc = any(sd['label']=='tower_tucohy' for sd in json_file['shapes']) 
            if bool_tuc:
                df.at[apples_indices,'tower_tucohy']= 1
            else:
                df.at[apples_indices,'tower_tucohy']= 0
######################################################################################################
#####################################################################################################                  

#create column containing info for all of the towers
df['towers'] = df[['tower_wooden', 'tower_lattice','tower_tucohy']].values.tolist()
df['tower_labels'] = np.empty((len(df), 0)).tolist()
for i in range(len(df)):

    if df['towers'][i][0] == 1:
        df['tower_labels'][i].append("tower_wooden")

    if df['towers'][i][1] == 1:
        df['tower_labels'][i].append("tower_lattice")

    if df['towers'][i][2] == 1:
        df['tower_labels'][i].append("tower_tucohy")

  ######################################################################################################
#####################################################################################################      
df['tower_wooden'] = df['tower_wooden'].astype(int)
df['tower_lattice'] = df['tower_lattice'].astype(int)
df['tower_tucohy'] = df['tower_tucohy'].astype(int)
#need to split the data into testing, training, and validating
random.seed(123)
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])    



In [ ]:
# the y data
y = np.array(train.drop(['Id', 'labels','towers','tower_labels'],axis=1))

In [ ]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# Currently, have only done the feature extraction for training images.  
this is in the below cell

In [ ]:
train_image = []
#this is the one that worked!!!
for i in range(train.shape[0]):
    # load an image from file
    image = load_img('C:/Users/miapi/Downloads/ttpla_dataset/sized_data/'+train.iloc[i][0], target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get extracted features
    features = model.predict(image)
    # save to file
    train_image.append(features)
feats = np.array(train_image)


this writes the data into a file and also reads the file
I also have code that writes each feature vector into the file individually - I think for formatting might need to write each feature vector into its own file?

In [ ]:
#cats is made by adding numpy array of vectors to pickle file
filename = 'cats'
outfile = open(filename,'wb')
dump(feats, outfile)
outfile.close()


infile = open(filename,'rb')
new_cat = pickle.load(infile)
infile.close()

In [ ]:
new_cat